In [45]:
import astroprov
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache
from astroquery.gaia import Gaia
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from bs4 import BeautifulSoup
import urllib2

In [2]:
clear_download_cache()

In [35]:
imDir = '/data/mj1e16/kepler/cal/'

In [2]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
external.external.apassdr9
external.external.gaiadr2_geometric_distance
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.tmass_xsc
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
public.public.dual
tap_config.tap_config.coord_sys
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighb

In [3]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [4]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [5]:
def queryObjectRegion(outputTable,searchRadius=5*u.arcsecond):
    imageCoords = []
    r = searchRadius
    
#     if inCat == 'dao':
#         xlabel = 'xcentroid'
#         ylabel = 'ycentroid'
#         label = 'id'
        
#     if inCat == 'sex':
#         xlabel = 'X_IMAGE'
#         ylabel = 'Y_IMAGE'
#         label = 'NUMBER'
        
    xlabel = 'RA'
    ylabel = 'DEC'
    label = 'NUMBER'
    
    flux = outputTable['FLUX_BEST']   
    ra = outputTable['RA']
    dec = outputTable['DEC']
    tableLength = len(outputTable)
    xcoord = outputTable['X_IMAGE']
    ycoord = outputTable['Y_IMAGE']
    imageCoords = [[x,y] for x,y in zip(xcoord,ycoord)]
    imageNames = outputTable['DIFF_NAME']
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype','sp')
    customSimbad.remove_votable_fields('coordinates')
    
    values = [['-'],[0],[0],['-'],['-'],[0],
              [0],[0],['-'],[0]]
    print(len(values))
    Names = ['MAIN_ID','RA_d','DEC_d','OTYPE','CLASS','X_IMAGE',
             'Y_IMAGE','flux','IMAGE','index']
    types = ['object','object','float64','object','object','float64',
             'float64','float64','object','int']     
    #result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-'],[0]], names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE','index'),dtype=('object','float64','float64','object','float64','float64','float64','object','object'))
    result_table = Table(values,names=Names,dtype=types)

    for x in range(len(imageCoords)):
        c = SkyCoord(ra[x],dec[x],frame='icrs',unit='deg')
        rt = customSimbad.query_region(c,radius=r)
        try:
            c = SkyCoord(ra[x],dec[x],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            #print(rt)
            rowlist = []
            
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageNames[x])
            rowlist.append(x)
            result_table.add_row(rowlist)

        except:
            #print('searchfailed')
            #result_table.add_row(['N/A',ra[x],dec[x],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageNames[x],x])
            result_table.add_row(['N/A',ra[x],dec[x],'Object Not Found','0',[imageCoords[x][0]],
                                  [imageCoords[x][1]],flux[x],imageNames[x],x],
                                 mask=[False,False,False,False,True,False,
                                       False,False,False,False])
    
    return result_table

In [6]:
def keywordFunction(tab,keywordList):
    names = tab.colnames
    #print(names)
    goodColNames = {}
    values = []
    for keyword in keywordList:
        goodColName = [x for x in names if x.find(keyword) != -1]
        if len(goodColName) != 0:
            #print(goodColName)
            #ravalues = []
            for x in range(len(tab)):
                values.append(tab[goodColName[0]][x])
            break
    return values

In [7]:
def vizierSearch(tab,cat):
    result_table = []
    indexref = 'none'
    indexes = []
    for x in range(len(tab)):
        index = tab['OBJECT INDEX'][x]
        if index != indexref:
            RA = tab['RA'][x]
            DEC = tab['DEC'][x]
            #print(bigtab['RA_d'][x],bigtab['DEC_d'][x])
            c = SkyCoord(ra=RA*u.deg,dec=DEC*u.deg)
            result_table_list = Vizier.query_region(c,radius=2*u.arcsec,catalog=cat)
            if len(result_table_list) != 0:
                indexes.append(index)
                if len(result_table) == 0:
                    #result_table_list = Vizier.query_region(c,radius=2*u.arcsec,catalog=cat)
                    result_table = result_table_list[0][0]
                else:
                    res_tab = result_table_list[0][0]
                    result_table = vstack([result_table,res_tab])
        indexref = index
        
#     aa = Column([], name='aa')
#     t.add_column(aa, index=0)
    if len(result_table) != 0:
        result_table['index'] = indexes
        return result_table
    else:
        return 'none'

In [8]:
def findSeparation(ra1,dec1,ra2,dec2):
    c1 = SkyCoord(ra=ra1*u.degree,dec=dec1*u.degree)
    c2 = SkyCoord(ra=ra2*u.degree,dec=dec2*u.degree)
    sep = c1.separation(c2)
    separc = sep.arcsec
    return separc

In [9]:
def getTabValues(Tab,index,tabType,origra,origdec):
    rowlist = 'none'
    if tabType == 'simbad':
        otypename = 'OTYPE'
        source = 'SIMBAD'
    if tabType == 'vizier':
        source = 'VIZIER'
        otypename = 'CLASS'
    Tab = Tab[1:]
    df = Tab.to_pandas()
    df2 = df[df[otypename] != 'Object Not Found']
    df3 = df2[df2['index'] == index]
    if len(df3) != 0:
        ra = df3['RA_d'].tolist()
        dec = df3['DEC_d'].tolist()
        otype = df3[otypename].tolist()
        separations = [round(findSeparation(ra1,dec1,origra,origdec),3) for ra1,dec1 in zip(ra,dec)]
        rowlist = [[index,source,0,0,x,y,0,0,'magfilter',sep,z] for x,y,z,sep in zip(ra,dec,otype,separations)]
    return rowlist

In [10]:
def generateRow(Tab,index,origra,origdec,Names,tabType):
    rowlist = 'none'
    columnNames = Tab.colnames
    Tab = Tab[1:]
    df = Tab.to_pandas()
    if tabType == 'simbad':
        df2 = df[df['OTYPE'] != 'Object Not Found']
    else:
        df2 = df
    df3 = df2[df2['index'] == index]
    rows = []
    astroTab = Table.from_pandas(df3)
    #print('tab',index,len(astroTab))
    if len(df3) != 0:
        for x in range(len(astroTab)):
            if tabType != 'simbad':
                ra1 = astroTab['RAJ2000'][x]
                dec1 = astroTab['DEJ2000'][x]
            else:
                ra1 = astroTab['RA_d'][x]
                dec1 = astroTab['DEC_d'][x]
            sep = round(findSeparation(ra1,dec1,origra,origdec),3)
            rowlist = [astroTab[name][x] if name in columnNames else '--' for name in Names]
            num = Names.index('OBJECT INDEX')
            rowlist[num] = index
            num = Names.index('SOURCE')
            rowlist[num] = tabType
            num = Names.index('SEPARATION')
            rowlist[num] = sep # linked to order
            rows.append(rowlist)
        #print('rows',index,len(rows))
    if len(rows) != 0:
        return rows
    else:
        return 'none'

In [11]:
def bigTable(origTab,simbadTab,ps1Tab,wiseTab,lamostTab,commies,epochs):
    origra = origTab['RA'].tolist()
    origdec = origTab['DEC'].tolist()
    origImage = origTab['IMAGE_NAME'].tolist()
    xim = origTab['X_IMAGE']
    yim = origTab['Y_IMAGE']
    origmag = origTab['MAG_BEST']
    magDiff = origTab['magDiff']
    searchTables = [simbadTab,ps1Tab,wiseTab,lamostTab]
    searchTabNames = ['simbad','PS1','AllWiSE','LAMOST']
    #result_table = Table([[0],[0],['-'],['-'],[0],[0],[0],['-'],[0]],names=('RA_d','DEC_d','Vmag','CLASS','xcentroid','ycentroid','flux','IMAGE','index'),dtype=('float64','float64','object','object','float64','float64','float64','object','int'))
    for x in range(len(origra)):
        if x == 0:

            namelist = ['OBJECT INDEX','SOURCE','ObjID','X_IMAGE','Y_IMAGE','RA_d','DEC_d','MAG','MAG_DIFF','No of Epochs','Comments',
                        'OTYPE','CLASS',
                        'magType','SEPARATION','pmRA','pmDE','var','z','Nd','Ns','Epoch','ex','Qual',
                        'W1mag','W2mag', 'W3mag', 'W4mag',
                        'Kmag','gKmag','rKmag', 'iKmag','zKmag', 'yKmag',   
                        'gmag','rmag','imag','zmag','ymag','Jmag', 'Hmag',
                        'snrg', 'snri','snru',   'snrz',          
                        'Target',  'AllWISE']
#             types = ['int','object','float64','float64','float64','float64','float64','float64','object','object','float64',
#                      'object','object','object','object','object','object','object','object',
#                     'object','object','object','object','object','object','object','object']
            types = ['object']*len(namelist)
            rowlistOG = [[x],[origImage[x]],['-'],[xim[x]],[yim[x]],[origra[x]],[origdec[x]],[origmag[x]],[magDiff[x]],[epochs[x]],[comments[x]]]
            difflen = len(namelist)-len(rowlistOG)
            extras = [['--'] for _ in range(difflen)]
            rowlistOG.extend(extras)

            result_table = Table(rowlistOG,names=namelist,dtype=types)
            for tab in range(len(searchTables)):
                rowlist = generateRow(searchTables[tab],x,origra[x],origdec[x],namelist,searchTabNames[tab])
                if rowlist != 'none':
                    for row in rowlist:
                        result_table.add_row(row)
                        print('working')
        else:
            rowlistOG = [x,origImage[x],'-',xim[x],yim[x],origra[x],origdec[x],origmag[x],magDiff[x],epochs[x],comments[x]]
            difflen = len(namelist)-len(rowlistOG)
            extras = ['--' for _ in range(difflen)]
            rowlistOG.extend(extras)
            result_table.add_row(rowlistOG)
            for tab in range(len(searchTables)):
                rowlist = generateRow(searchTables[tab],x,origra[x],origdec[x],namelist,searchTabNames[tab])
                if rowlist != 'none':
                    for row in rowlist:
                        result_table.add_row(row)
    return result_table

In [12]:
def getPanStarCutOut(ra,dec):
    url = 'https://ps1images.stsci.edu/cgi-bin/ps1cutouts?pos={}%2C{}&filter=color&filter=g&filter=r&filter=i&filter=z&filter=y&filetypes=stack&auxiliary=data&size={}&output_size=0&verbose=0&autoscale=99.500000&catlist='.format(ra,dec,size)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    th = soup.findAll('td')
    try:
        link = th[1]
        downLink = link.findAll('img')
        downCut = downLink[0]['src']
        loc1 = downCut.find('cgi-bin')
        loc2 = downCut.find('=/') + len('=/')
        if loc1 != -1:
            if loc2 != -1:
                newCut = downCut[:loc1] + downCut[loc2:]

        loc3 = newCut.find('&')
        smallCut = newCut[:loc3]
        http = 'https:'+smallCut
        print(http)
        subprocess.call(['wget','-O','/home/mj1e16/cutOuts/{}'.format(outname),http])
    except:
        print('WARNING - panstarrs image note found ra = {}, dec = {}'.format(ra,dec))


In [13]:
def XraySearch(tab):
    indexref = 'ooo'
    chandra = {}
    rosat = {}
    xmm = {}
    for x in range(len(tab)):
        index = tab['OBJECT INDEX'][x]
        if index != indexref:
            RA = tab['RA'][x]
            DEC = tab['DEC'][x]
            #print(tab['RA_d'][x],tab['DEC_d'][x])
            c = SkyCoord(ra=RA*u.deg,dec=DEC*u.deg)
            result_table1 = Vizier.query_region(c,radius=2*u.arcsec,catalog='Chandra')
            if len(result_table1) > 0:
                chandra[str(x)] = result_table1
                print('Chandra',result_table1,x)
            #print(result_table[0])
            result_table2 = Vizier.query_region(c,radius=10*u.arcsec,catalog='Rosat')
            if len(result_table2) > 0:
                rosat[str(x)] = result_table2
                print('Rosat',result_table2,x)
            #print(result_table[0])
            result_table3 = Vizier.query_region(c,radius=10*u.arcsec,catalog='XMM-Newton')
            if len(result_table3) > 0:
                xmm[str(x)] = result_table3
                print('XMM',result_table3,x)
    #             for y in range(len(result_table3)):
    #                 print(result_table3[y])
        indexref = tab['OBJECT INDEX'][x]
    return([chandra,rosat,xmm])

In [101]:
vizmatch = ascii.read('/home/mj1e16/outTablesImproved/matched15magViz.csv')
customSimbad = Simbad()
customSimbad.add_votable_fields('ra(d)','dec(d)','otype','sp')
ra = vizmatch['RA'].tolist()
dec= vizmatch['DEC'].tolist()
bigtab = []
for x in range(len(ra)):
    c = SkyCoord(ra[x],dec[x],frame='icrs',unit='deg')
    rt = customSimbad.query_region(c,radius=2*u.arcsec)
    #simbadSearch = queryObjectRegion(vizmatch,searchRadius=2*u.arcsec)
    if len(bigtab) == 0:
        bigtab = simbadSearch
    else:
        bigtab = vstack([bigtab,simbadSearch])

In [102]:
bigtab

MAIN_ID,RA_d,DEC_d,OTYPE,CLASS,X_IMAGE,Y_IMAGE,flux,IMAGE,index
object,object,float64,object,object,float64,float64,float64,object,int64
-,0,0.0,-,-,0.0,0.0,0.0,-,0
N/A,285.6831915871575,43.045040272922726,Object Not Found,--,342.2666,242.125,1904.898,diff_imNo11_channel48_ccd24.fits,0
N/A,288.3913293514153,46.043645716417046,Object Not Found,--,683.0833,458.6268,5891.487,diff_imNo11_channel22_ccd38.fits,1
N/A,291.9081476036849,47.058497168533194,Object Not Found,--,429.9972,24.7407,4972.103,diff_imNo11_channel18_ccd58.fits,2
N/A,281.75827968509583,46.712644169501274,Object Not Found,--,318.6326,850.0745,2883.682,diff_imNo15_channel10_ccd14.fits,3
N/A,281.60764434985947,47.49186803042227,Object Not Found,--,695.7756,712.7224,1964.376,diff_imNo15_channel11_ccd15.fits,4
N/A,283.1430488833758,43.8178082207455,Object Not Found,--,872.8135,244.0135,2041.174,diff_imNo15_channel30_ccd2.fits,5
N/A,285.68907608826987,48.721619060900146,Object Not Found,--,748.9758,808.0461,1715.738,diff_imNo15_channel6_ccd34.fits,6
N/A,287.9131858266935,50.43142296281254,Object Not Found,--,138.0509,863.371,3124.981,diff_imNo15_channel3_ccd55.fits,7


In [14]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/matchplus3Magdiff.csv')

In [15]:
kepler = otherGaiaTab #ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')
indexes = range(len(kepler))
kepler['OBJECT INDEX'] = indexes
xrayobs = XraySearch(kepler)

In [16]:
print(len(xrayobs))

3


In [50]:
df = simbadSearch.to_pandas()
df2 = df[df['OTYPE'] != 'Object Not Found']
df2

,MAIN_ID,RA_d,DEC_d,OTYPE,CLASS,X_IMAGE,Y_IMAGE,flux,IMAGE,index
0,-,0,0.0,-,-,0.0,0.0,0.0,-,0.0


In [56]:
ps1

RAJ2000,DEJ2000,objID,f_objID,Qual,e_RAJ2000,e_DEJ2000,Epoch,Ns,Nd,gmag,e_gmag,gKmag,e_gKmag,gFlags,rmag,e_rmag,rKmag,e_rKmag,rFlags,imag,e_imag,iKmag,e_iKmag,iFlags,zmag,e_zmag,zKmag,e_zKmag,zFlags,ymag,e_ymag,yKmag,e_yKmag,yFlags,index
deg,deg,,,,arcs,arcs,d,,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,
float64,float64,int64,int64,int16,float64,float64,float64,int16,int16,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,int64
281.204444750,47.118410960,164542812045092819,436527104,52,0.0115,0.0193,56207.713125000,5,52,22.0335,0.0681,22.4095,0.0458,115000,21.0005,0.0314,21.0652,0.0903,115000,20.2086,0.0281,20.3279,0.0462,115000,19.7718,0.0415,19.9713,0.0792,115000,19.5148,0.0402,19.4774,0.1286,115000,10
282.277557220,48.768685480,166522822775012991,436527104,52,0.0572,0.0485,56052.143715280,3,14,--,--,--,--,16416,--,--,--,--,16416,21.6356,0.0580,21.5729,0.0842,115000,20.8544,0.0505,20.9447,0.1995,115000,20.0290,0.1825,19.9527,0.3104,114992,20
280.556655240,43.236719390,159882805568064704,436527104,52,0.0319,0.0172,56179.739085650,5,61,--,--,--,--,114720,21.5555,0.0340,21.5085,0.0499,115000,20.3774,0.0146,20.3404,0.0485,115000,19.8638,0.0611,19.9487,0.1402,115000,19.4679,0.0348,19.3652,0.0935,115000,23
280.316932490,43.811652100,160572803168584606,436527104,52,0.0289,0.0676,56082.313321760,4,33,21.9816,0.0824,22.0939,0.1848,115000,21.6983,0.0768,21.7782,0.0778,115000,21.5314,0.0839,21.4824,0.0755,115000,21.0892,0.0983,21.4142,0.0488,115000,--,--,--,--,16416,26


In [57]:
wise

AllWISE,RAJ2000,DEJ2000,Im,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,ccf,ex,var,pmRA,e_pmRA,pmDE,e_pmDE,qph,d2M,_2M,index
,deg,deg,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,mas / yr,mas / yr,mas / yr,mas / yr,,arcs,,
str19,float64,float64,str2,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str4,uint8,str4,int32,int32,int32,int32,str4,float32,str2,int64
J184449.06+470706.9,281.2044187,47.1185871,Im,18.278,0.201,18.003,--,12.902,--,9.568,--,--,--,--,--,--,--,0000,0,nnnn,184,2475,-60,2681,BUUU,--,2M,10
J184213.57+431411.4,280.5565792,43.2365190,Im,17.109,0.082,16.853,0.230,13.059,--,8.914,--,--,--,--,--,--,--,dd00,1,nnnn,-272,912,189,992,ABUU,--,2M,23


In [58]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/matchSNabove10.csv')

In [59]:
indexes = range(len(otherGaiaTab))
otherGaiaTab['OBJECT INDEX'] = indexes

In [17]:
simbadSearch = queryObjectRegion(otherGaiaTab,searchRadius=2*u.arcsec)

10
INFO: Upgrading Table to masked Table. Use Table.filled() to convert to unmasked table. [astropy.table.table]


In [18]:
print(len(simbadSearch))

90


In [23]:
df = simbadSearch.to_pandas()
df2 = df[df['MAIN_ID'] != 'N/A']
df2

,MAIN_ID,RA_d,DEC_d,OTYPE,CLASS,X_IMAGE,Y_IMAGE,flux,IMAGE,index
0,-,0,0.0,-,-,0.0,0.0,0.0,-,0.0


In [24]:
ps1 = vizierSearch(otherGaiaTab,'PS1')
print(len(ps1))

4


In [25]:
ps1

RAJ2000,DEJ2000,objID,f_objID,Qual,e_RAJ2000,e_DEJ2000,Epoch,Ns,Nd,gmag,e_gmag,gKmag,e_gKmag,gFlags,rmag,e_rmag,rKmag,e_rKmag,rFlags,imag,e_imag,iKmag,e_iKmag,iFlags,zmag,e_zmag,zKmag,e_zKmag,zFlags,ymag,e_ymag,yKmag,e_yKmag,yFlags,index
deg,deg,,,,arcs,arcs,d,,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,mag,mag,mag,mag,,
float64,float64,int64,int64,int16,float64,float64,float64,int16,int16,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,float64,float32,float64,float32,int32,int64
281.204444750,47.118410960,164542812045092819,436527104,52,0.0115,0.0193,56207.713125000,5,52,22.0335,0.0681,22.4095,0.0458,115000,21.0005,0.0314,21.0652,0.0903,115000,20.2086,0.0281,20.3279,0.0462,115000,19.7718,0.0415,19.9713,0.0792,115000,19.5148,0.0402,19.4774,0.1286,115000,10
282.277557220,48.768685480,166522822775012991,436527104,52,0.0572,0.0485,56052.143715280,3,14,--,--,--,--,16416,--,--,--,--,16416,21.6356,0.0580,21.5729,0.0842,115000,20.8544,0.0505,20.9447,0.1995,115000,20.0290,0.1825,19.9527,0.3104,114992,20
280.556655240,43.236719390,159882805568064704,436527104,52,0.0319,0.0172,56179.739085650,5,61,--,--,--,--,114720,21.5555,0.0340,21.5085,0.0499,115000,20.3774,0.0146,20.3404,0.0485,115000,19.8638,0.0611,19.9487,0.1402,115000,19.4679,0.0348,19.3652,0.0935,115000,23
280.316932490,43.811652100,160572803168584606,436527104,52,0.0289,0.0676,56082.313321760,4,33,21.9816,0.0824,22.0939,0.1848,115000,21.6983,0.0768,21.7782,0.0778,115000,21.5314,0.0839,21.4824,0.0755,115000,21.0892,0.0983,21.4142,0.0488,115000,--,--,--,--,16416,26


In [20]:
wise = vizierSearch(otherGaiaTab,'AllWiSE')
print(len(wise))

2


In [26]:
wise

AllWISE,RAJ2000,DEJ2000,Im,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,ccf,ex,var,pmRA,e_pmRA,pmDE,e_pmDE,qph,d2M,_2M,index
,deg,deg,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,mas / yr,mas / yr,mas / yr,mas / yr,,arcs,,
str19,float64,float64,str2,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str4,uint8,str4,int32,int32,int32,int32,str4,float32,str2,int64
J184449.06+470706.9,281.2044187,47.1185871,Im,18.278,0.201,18.003,--,12.902,--,9.568,--,--,--,--,--,--,--,0000,0,nnnn,184,2475,-60,2681,BUUU,--,2M,10
J184213.57+431411.4,280.5565792,43.2365190,Im,17.109,0.082,16.853,0.230,13.059,--,8.914,--,--,--,--,--,--,--,dd00,1,nnnn,-272,912,189,992,ABUU,--,2M,23


In [21]:
LAMOST = vizierSearch(otherGaiaTab,'LAMOST')
print(len(LAMOST))

4


In [27]:
LAMOST

'none'

In [42]:
#var = vizierSearch(bigtab,'J/AJ/156/241/')

In [46]:
comments = ['Very Elongated, Likely noise','Probable Transient','Probable Transient','Easily visible counterpart',
            'Easily visible counterpart','Easily visible counterpart','Easily visible counterpart','Easily visible counterpart',
           't shaped, possibly noise','Likely noise','Easily visible counterpart','Easily visible counterpart',
            'Strange shape, likely noise','Strange shape, likely noise','Probable transient','Counterpart visible in median image but not in PanSTARRS',
           'Possible transient, chance of a visible counterpart','Probable transient','Strange shape, likely noise',
           'Saturated counterpart','Probable transient','Easily visible counterpart','Very elongated, probably noise',
           'Easily visible counterpart','Probable transient','probable transient','Easily visible counterpart','Possible transient',
           'Probable transient','Probable transient','Easily visible counterpart','Easily visible counterpart','Easily visible counterpart',
           'Possible transient','Probable transient','Easily visible counterpart']

In [29]:
comments = ['']*len(otherGaiaTab)

In [47]:
epochs = [1, 1, 1, 52, 52, 52, 52, 52, 1, 1, 14, 51, 1, 1, 1, 1, 1, 1, 1, 41, 1, 1, 1, 52, 1, 1, 52, 1, 1, 1, 52, 52, 52, 1, 1, 3]

In [31]:
epochs = ['']*len(otherGaiaTab)

In [52]:
emptyTab = Table.from_pandas(df2)

In [53]:
simbadSearch = emptyTab
LAMOST = emptyTab

In [47]:
comments = ['--' for x in range(len(otherGaiaTab))]
epochs =  [1 for x in range(len(otherGaiaTab))]

In [32]:
newBigTab = bigTable(otherGaiaTab,simbadSearch,ps1,wise,LAMOST,comments,epochs)

AttributeError: 'str' object has no attribute 'colnames'

In [ ]:
ascii.write(newBigTab,'/home/mj1e16/outTablesImproved/newbigtabSN10.csv',format='csv',fast_writer=False)


In [55]:
newBigTab

OBJECT INDEX,SOURCE,ObjID,X_IMAGE,Y_IMAGE,RA_d,DEC_d,MAG,MAG_DIFF,No of Epochs,Comments,OTYPE,CLASS,magType,SEPARATION,pmRA,pmDE,var,z,Nd,Ns,Epoch,ex,Qual,W1mag,W2mag,W3mag,W4mag,Kmag,gKmag,rKmag,iKmag,zKmag,yKmag,gmag,rmag,imag,zmag,ymag,Jmag,Hmag,snrg,snri,snru,snrz,Target,AllWISE
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
0,kplr2009114174833_ffi-cal.fits[24],-,342.2666,242.125,285.683191587,43.0450402729,16.8003,-3.0803900093307286,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
1,kplr2009114174833_ffi-cal.fits[38],-,683.0833,458.6268,288.391329351,46.0436457164,15.5744,-3.817542693056524,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
2,kplr2009114174833_ffi-cal.fits[58],-,429.9972,24.7407,291.908147604,47.0584971685,15.7586,-4.733674110651155,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
3,kplr2009114204835_ffi-cal.fits[14],-,318.6326,850.0745,281.758279685,46.7126441695,16.3501,-5.658338521792768,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
4,kplr2009114204835_ffi-cal.fits[15],-,695.7756,712.7224,281.60764435,47.4918680304,16.7669,-3.605746252562367,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
5,kplr2009114204835_ffi-cal.fits[2],-,872.8135,244.0135,283.143048883,43.8178082207,16.7253,-5.278683792538309,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
6,kplr2009114204835_ffi-cal.fits[34],-,748.9758,808.0461,285.689076088,48.7216190609,16.9139,-3.1348216901835055,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
7,kplr2009114204835_ffi-cal.fits[55],-,138.0509,863.371,287.913185827,50.4314229628,16.2629,-3.230583337813577,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
8,kplr2009114204835_ffi-cal.fits[5],-,392.062,72.952,284.837718718,42.5143291681,16.7641,-3.748741535732072,1,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [33]:
otherGaiaTab

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME,index,magDiff,MAG_BEST_ORIG,S/N,OBJECT INDEX
int64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32,int64,float64,float64,float64,int64
158,1904.898,13.55821,16.8003,342.2666,242.125,0.056,285.6831915871575,43.045040272922726,kplr2009114174833_ffi-cal.fits[24],meidian_48_24.fits,diff_imNo11_channel48_ccd24.fits,474785,-3.0803900093307286,16.7348,26.49346430063822,0
156,5891.487,17.17145,15.5744,683.0833,458.6268,0.315,288.3913293514153,46.043645716417046,kplr2009114174833_ffi-cal.fits[38],meidian_22_38.fits,diff_imNo11_channel22_ccd38.fits,182298,-3.817542693056524,15.5417,45.4409314217967,1
0,4972.103,37.4395,15.7586,429.9972,24.7407,0.351,291.9081476036849,47.058497168533194,kplr2009114174833_ffi-cal.fits[58],meidian_18_58.fits,diff_imNo11_channel18_ccd58.fits,122511,-4.733674110651155,15.7447,41.14911261412415,2
43,2883.682,16.82505,16.3501,318.6326,850.0745,0.208,281.75827968509583,46.712644169501274,kplr2009114204835_ffi-cal.fits[14],meidian_10_14.fits,diff_imNo15_channel10_ccd14.fits,77535,-5.658338521792768,16.3442,31.14521791542877,3
62,1964.376,13.94542,16.7669,695.7756,712.7224,0.061,281.60764434985947,47.49186803042227,kplr2009114204835_ffi-cal.fits[15],meidian_11_15.fits,diff_imNo15_channel11_ccd15.fits,1960,-3.605746252562367,16.727,26.383454066966614,4
164,2041.174,15.32619,16.7253,872.8135,244.0135,0.046,283.1430488833758,43.8178082207455,kplr2009114204835_ffi-cal.fits[2],meidian_30_2.fits,diff_imNo15_channel30_ccd2.fits,350703,-5.278683792538309,16.7169,26.253586003493012,5
61,1715.738,13.67365,16.9139,748.9758,808.0461,0.092,285.68907608826987,48.721619060900146,kplr2009114204835_ffi-cal.fits[34],meidian_6_34.fits,diff_imNo15_channel6_ccd34.fits,318749,-3.1348216901835055,16.8516,25.080565894018715,6
42,3124.981,17.25937,16.2629,138.0509,863.371,0.103,287.9131858266935,50.43142296281254,kplr2009114204835_ffi-cal.fits[55],meidian_3_55.fits,diff_imNo15_channel3_ccd55.fits,375749,-3.230583337813577,16.206,33.70933099507585,7
62,1969.503,13.00574,16.7641,392.062,72.952,0.031,284.83771871752884,42.51432916809274,kplr2009114204835_ffi-cal.fits[5],meidian_49_5.fits,diff_imNo15_channel49_ccd5.fits,272367,-3.748741535732072,16.7292,26.316969094772997,8


In [96]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/match1mag.csv')

imNames = otherGaiaTab['IMAGE_NAME'].tolist()
RA = otherGaiaTab['RA'].tolist()
DEC = otherGaiaTab['DEC'].tolist()
variables = []
for x in range(len(imNames)):
    loc1 = imNames[x].find('[')
    loc2 = imNames[x].find(']')
    ext = int(imNames[x][loc1+1:loc2])
    #print(ext)
    data = fits.open(imDir+imNames[x][:loc1])
    date = data[ext].header['MJDSTART']
    endDate = data[ext].header['MJDEND']
    #datefull.append(date)
    variables.append([RA[x],DEC[x],date+15,endDate-15])# don't know about this order...
#print(min(datefull),max(datefull))

for x in range(len(variables)):
    vizierImageSearch(test_uri,variables[x])
    time.sleep(1)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [90]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/match25mag.csv')

In [ ]:
allResults = 

In [91]:
def vizierImageSearch(uri,variables):
    locators = ['285.6831915871575','+43.045040272922726','+54955.74206','+44935.72162632']
    for x in range(len(locators)):
        loc1 = uri.find(locators[x])
        loc2 = loc1 + len(locators[x])
        newuri = uri[:loc1]+str(variables[x])+uri[loc2:]
        uri = newuri
    test_return = requests.get(uri)
    columns = [col.get('description') for col in test_return.json().get('metadata')]
    df = pd.DataFrame(test_return.json().get('data'), columns=columns)
    print(len(df))
    return df

In [87]:
test_uri = "http://cdsarc.u-strasbg.fr/saadavizier.tap/tap/sync?query=SELECT+TOP+500+*+FROM+obscore+WHERE+1=CONTAINS(POINT(%27ICRS%27,s_ra,s_dec),CIRCLE(%27ICRS%27,285.6831915871575,+43.045040272922726,15))+++++AND+t_min+%3C=+54955.74206+++++AND+t_max+%3E=+44935.72162632&phase=RUN&lang=adql&request=doQuery&format=json"

In [92]:
for x in range(len(variables)):
    vizierImageSearch(test_uri,variables[x])
    time.sleep(1)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
otherGaiaTab = ascii.read('/home/mj1e16/outTablesImproved/match15mag.csv')
for x in range(len(variables)):
    vizierImageSearch(test_uri,variables[x])
    time.sleep(1)

In [89]:
a = vizierImageSearch(test_uri,variables[0])
a

0


,Data product,Calibration level,Astronomical object observed,Observation ID,Name of the data collection,Dataset identifier given by the publisher,URL used to access (download) dataset,File content format,Estimated size of dataset in kilo bytes,"Central right ascension, ICRS",...,Stop in spectral coordinates,Spectral band,UCD of observable,List of polarization states,Name of the facility used for this observation,Name of the instrument used for this observation,internal Saada OID (non persistent),WCS detected,Extension (HDU),Bibliography reference


In [75]:
df = vizierImageSearch(test_uri,['285.6831915871575','+43.045040272922726','+44935.72162632','+54955.74206'])

168


In [76]:
df

,Data product,Calibration level,Astronomical object observed,Observation ID,Name of the data collection,Dataset identifier given by the publisher,URL used to access (download) dataset,File content format,Estimated size of dataset in kilo bytes,"Central right ascension, ICRS",...,Stop in spectral coordinates,Spectral band,UCD of observable,List of polarization states,Name of the facility used for this observation,Name of the instrument used for this observation,internal Saada OID (non persistent),WCS detected,Extension (HDU),Bibliography reference
0,spectrum,-1,V533Her,V533Her_IUE.fits,J/A+A/560/A49,ivo://CDS.VizieR/J/A+A/560/A49?res=V533Her_IUE...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,11520,273.585333,...,3.000000e-07,UV,obs,NotSet,None,IUE,1153204658915770391,4,None,2013A&A...560A..49S
1,spectrum,-1,beta Lyr (ARVIpec),pec_betaLyr_2_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.521875,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051470,0,None,2010AJ....140.1758T
2,spectrum,-1,beta Lyr (ARVIpec),pec_betaLyr_2.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.521875,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051469,0,None,2010AJ....140.1758T
3,spectrum,-1,beta Lyr,pec_betaLyr_1_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.539125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051468,0,None,2010AJ....140.1758T
4,spectrum,-1,beta Lyr,pec_betaLyr_1.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.539125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051467,0,None,2010AJ....140.1758T
5,spectrum,-1,HD 184905,pec_V1264Cyg_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_V1264Cy...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,408960,293.656333,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051376,0,None,2010AJ....140.1758T
6,spectrum,-1,HD 184905,pec_V1264Cyg.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_V1264Cy...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,408960,293.656333,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051375,0,None,2010AJ....140.1758T
7,spectrum,-1,RR Lyr,pec_RRLyr_2_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_2...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,291.316042,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051301,0,None,2010AJ....140.1758T
8,spectrum,-1,RR Lyr,pec_RRLyr_2.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_2...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,423360,291.316042,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051299,0,None,2010AJ....140.1758T
9,spectrum,-1,RR Lyr (ARVIpec),pec_RRLyr_1_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_1...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,291.366125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051297,0,None,2010AJ....140.1758T


In [57]:
test_return = requests.get(test_uri)

In [65]:
columns = [col.get('description') for col in test_return.json().get('metadata')]

In [66]:
pd.DataFrame(test_return.json().get('data'), columns=columns)

,Data product,Calibration level,Astronomical object observed,Observation ID,Name of the data collection,Dataset identifier given by the publisher,URL used to access (download) dataset,File content format,Estimated size of dataset in kilo bytes,"Central right ascension, ICRS",...,Stop in spectral coordinates,Spectral band,UCD of observable,List of polarization states,Name of the facility used for this observation,Name of the instrument used for this observation,internal Saada OID (non persistent),WCS detected,Extension (HDU),Bibliography reference
0,spectrum,-1,V533Her,V533Her_IUE.fits,J/A+A/560/A49,ivo://CDS.VizieR/J/A+A/560/A49?res=V533Her_IUE...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,11520,273.585333,...,3.000000e-07,UV,obs,NotSet,None,IUE,1153204658915770391,4,None,2013A&A...560A..49S
1,spectrum,-1,beta Lyr (ARVIpec),pec_betaLyr_2_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.521875,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051470,0,None,2010AJ....140.1758T
2,spectrum,-1,beta Lyr (ARVIpec),pec_betaLyr_2.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.521875,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051469,0,None,2010AJ....140.1758T
3,spectrum,-1,beta Lyr,pec_betaLyr_1_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.539125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051468,0,None,2010AJ....140.1758T
4,spectrum,-1,beta Lyr,pec_betaLyr_1.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_betaLyr...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,282.539125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051467,0,None,2010AJ....140.1758T
5,spectrum,-1,HD 184905,pec_V1264Cyg_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_V1264Cy...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,408960,293.656333,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051376,0,None,2010AJ....140.1758T
6,spectrum,-1,HD 184905,pec_V1264Cyg.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_V1264Cy...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,408960,293.656333,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051375,0,None,2010AJ....140.1758T
7,spectrum,-1,RR Lyr,pec_RRLyr_2_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_2...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,291.316042,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051301,0,None,2010AJ....140.1758T
8,spectrum,-1,RR Lyr,pec_RRLyr_2.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_2...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,423360,291.316042,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051299,0,None,2010AJ....140.1758T
9,spectrum,-1,RR Lyr (ARVIpec),pec_RRLyr_1_vh.fits,J/AJ/140/1758,ivo://CDS.VizieR/J/AJ/140/1758?res=pec_RRLyr_1...,http://cdsarc.u-strasbg.fr/saadavizier/downloa...,application/fits,426240,291.366125,...,1.000000e-06,Optical,obs,NotSet,Mt. Ekar 182 cm. Telescope,Echelle Hi-Res Spectrograph,1153204753405051297,0,None,2010AJ....140.1758T
